<h2>Imports</h2>

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

<h2>Modeling</h2>

<h3>Predicting Pts Based Off Other Stats</h3>

In [3]:
players = pd.DataFrame()
player_pickles = os.listdir('player_stats')
for player_pickle in player_pickles:
    try:
        player = pd.read_pickle('player_stats/'+player_pickle)
        players = players.append(player)
    except:
        continue
# player.drop_duplicates(inplace=True)
players = players.reset_index()
players.dropna(inplace=True)
pts_target = ['PTS']
pts_features = ['Age','FGA','FG%','FTA','FT%','TRB','AST']
# print(players)
pts_X = players[pts_features]
pts_Y = players[pts_target]

# print(pts_X, '\n\n', pts_Y)
# print(pts_X)

In [4]:
# pts_X_train, pts_X_test, pts_Y_train, pts_Y_test = train_test_split(pts_X, pts_Y, test_size=(0.33), shuffle=False,random_state = 314)
pts_X_train, pts_X_test, pts_Y_train, pts_Y_test = train_test_split(pts_X, pts_Y, test_size=(0.33), shuffle=False,random_state = 434)


In [5]:
pts_linear = LinearRegression()
# pts_linear = DecisionTreeRegressor(max_depth=9)
# pts_linear = LogisticRegression()
pts_linear.fit(pts_X_train, pts_Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
mj = pd.read_pickle('player_stats/michael_jordan.pickle')
mj.reset_index(inplace=True)
pts_prediction = pts_linear.predict(pts_X_test)

In [7]:
print(sqrt(mean_squared_error(y_true=pts_prediction, y_pred=pts_Y_test)))


0.8362467798408505


<h3>Predicting Stats of Next Year</h3>

In [27]:
all_stats = pd.DataFrame()
years_to_predict = 1
for player_pickle in player_pickles:
    try:
        player_stat = pd.read_pickle('player_stats/'+player_pickle)
        player_stat[str(years_to_predict)+'-Year-PTS'] = player_stat['PTS'].shift(-years_to_predict)
        if all_stats.empty:
            all_stats = player_stat
            print('empty')
        else:
            all_stats = all_stats.append(player_stat)
    except:
        print(player_pickle)
all_stats.dropna(inplace=True)
print(len(all_stats))
print(all_stats.tail())
    
    

empty
.DS_Store
17236
        G  FGA   FG%  FTA   FT%  TRB  AST   PTS  1-Year-PTS
Age                                                        
25   48.0  4.6  49.1  1.5  55.6  5.0  0.8   5.3       6.800
26   72.0  5.6  55.1  1.1  61.8  5.3  0.9   6.8       7.200
27   79.0  5.9  48.7  1.9  58.1  5.0  1.1   7.2       5.900
28   79.0  4.9  49.0  1.3  67.0  5.7  1.1   5.9      10.000
29   71.0  8.3  51.6  1.5  67.3  8.0  1.9  10.0       6.625


In [28]:
goal_col = str(years_to_predict)+'-Year-Pts'
player_features, player_future_pts = all_stats.iloc[:,:-1], all_stats.iloc[:,-1]
print(player_features.head(),player_future_pts.head())


             G        FGA        FG%       FTA        FT%  TRB       AST  \
Age                                                                        
21   46.666667   9.633333  32.466667  3.433333  61.333333  8.6  1.333333   
27   44.000000  10.033333  26.666667  4.200000  69.133333  3.7  2.500000   
29   66.000000   4.800000  28.300000  1.800000  75.600000  3.7  1.500000   
22   13.000000   3.800000  46.900000  0.800000  70.000000  0.5  1.400000   
24   35.000000   3.800000  37.300000  0.800000  75.000000  0.6  2.100000   

          PTS  
Age            
21   8.666667  
27   8.300000  
29   4.100000  
22   5.000000  
24   3.700000   Age
21    10.0
27     4.1
29     2.3
22     3.7
24     7.2
Name: 1-Year-PTS, dtype: float64


In [44]:
x_train, x_test, Y_train, Y_test = train_test_split(player_features,player_future_pts,test_size=0.10)


In [45]:
predictor = LinearRegression(n_jobs=-1)
predictor.fit(x_train, Y_train)
accuracy = predictor.score(x_test,Y_test)

In [46]:
print(accuracy)

0.7049872129463466


<h5>Test</h5>

In [55]:
sc = pd.read_pickle('player_stats/stephen_curry.pickle')
sc[goal_col] = sc['PTS'].shift(-years_to_predict)
sc_features,sc_future_pts = sc.iloc[:,:-1], sc.iloc[:,-1].values.reshape(-1,1)
predictions = predictor.predict(sc_features).reshape(-1,1)
print(predictions,'\n\n\n', sc_future_pts)

[[15.87575067]
 [16.96275019]
 [13.30083732]
 [20.67547963]
 [21.79221424]
 [21.69533644]
 [27.3028417 ]
 [23.00547494]
 [24.3109234 ]
 [24.66749765]] 


 [[18.6]
 [14.7]
 [22.9]
 [24. ]
 [23.8]
 [30.1]
 [25.3]
 [26.4]
 [27.3]
 [ nan]]
